<a href="https://colab.research.google.com/github/rija-ansari/MSE1003H_RijaAnsari/blob/main/MSE1003_Assignment1_RA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

1. Research the dataset for this project.
2. Define the research question

In [1]:
import os
!pip install python-dotenv

In [2]:
!which python

/Users/rija/opt/anaconda3/bin/python


In [3]:
import dotenv 
print("dotenv imported successfully!")

dotenv imported successfully!


Using Python3.9.12 for correct dotenv installation

In [4]:
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv('MPI_KEY')


create a virtual environment
put in gitignore
push to github